In [2]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [3]:
import pdfplumber

In [4]:
import json

In [5]:
import re

In [6]:
def parse_resume(file):
    with pdfplumber.open(file) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()

    name = None
    email = None
    phone = None
    education = []
    work_experience = []

    # Name
    name_regex = r'^[A-Z][a-z]+\s+[A-Z][a-z]+$'
    name_match = re.search(name_regex, text)
    if name_match:
        name = name_match.group()

    # Email
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    email_match = re.search(email_regex, text)
    if email_match:
        email = email_match.group()

    # phone number
    phone_regex = r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'
    phone_match = re.search(phone_regex, text)
    if phone_match:
        phone = phone_match.group()

    # education
    education_regex = r'((?:Bachelor|Master|PhD)\s+of\s+[A-Za-z\s]+(?:\s+in\s+[A-Za-z\s]+)?)\s*(\d{4})\s*-?\s*(\d{4})?'
    for education_match in re.finditer(education_regex, text):
        degree, start_year, end_year = education_match.groups()
        education.append({'degree': degree, 'start_year': start_year, 'end_year': end_year})

    # work experience
    work_experience_regex = r'([A-Za-z\s]+)\s*-\s*([A-Za-z\s]+)?\s*\n\s*(?:([A-Za-z\s]+)\s*\n)?\s*([A-Za-z]{3,}\s+\d{4})\s*-\s*([A-Za-z]{3,}\s+\d{4})\s*\n\s*([A-Za-z\s,]+)?'
    for work_experience_match in re.finditer(work_experience_regex, text):
        company, title, location, start_date, end_date, description = work_experience_match.groups()
        work_experience.append({'company': company, 'title': title, 'location': location, 'start_date': start_date, 'end_date': end_date, 'description': description})

    # create JSON
    resume_data = {
        'name': name,
        'email': email,
        'phone': phone,
        'education': education,
        'work_experience': work_experience
    }

    return json.dumps(resume_data)

In [1]:
# print(parse_resume('/content/Elementary Education Teacher Resume.pdf'))